In [1]:
library(tidyverse) ## basics
library(survival) ## survival analysis
library(survminer) ## survival plotting

Registered S3 methods overwritten by 'tibble':
  method     from  
  format.tbl pillar
  print.tbl  pillar
Registered S3 method overwritten by 'rvest':
  method            from
  read_xml.response xml2
-- Attaching packages ------------------------------------------------------------------------------- tidyverse 1.2.1 --
v ggplot2 3.3.5       v purrr   0.3.2  
v tibble  2.1.1       v dplyr   0.8.0.1
v tidyr   0.8.3       v stringr 1.4.0  
v readr   1.3.1       v forcats 0.4.0  
-- Conflicts ---------------------------------------------------------------------------------- tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()
Warning message:
"package 'survminer' was built under R version 3.6.3"Loading required package: ggpubr
Warning message:
"package 'ggpubr' was built under R version 3.6.3"Error: package or namespace load failed for 'ggpubr' in loadNamespace(j <- i[[1L]], c(lib.loc, .libPaths()), versionCheck = vI[[j]]):
 namespace 'bro

ERROR: Error: package 'ggpubr' could not be loaded


In [3]:
# load metadata
metadata <- read.delim(r"(C:Users\Dasa\Documents\MEGA\IMG\study_5_revisions\qiime2_results\metadata.tsv)", sep='\t')

# load alpha diversity
shannon <- read.delim("/Users/dagmarschierova/MEGA/IMG/study_5_revisions/qiime2_results/diversity_raref_1192/alpha/shannon_tabulated.tsv", sep='\t')
faith <- read.delim("/Users/dagmarschierova/MEGA/IMG/study_5_revisions/qiime2_results/diversity_raref_1192/alpha/faith_pd_tabulated.tsv", sep='\t')
asv <- read.delim("/Users/dagmarschierova/MEGA/IMG/study_5_revisions/qiime2_results/diversity_raref_1192/alpha/observed_features_tabulated.tsv", sep='\t')
chao1 <- read.delim("/Users/dagmarschierova/MEGA/IMG/study_5_revisions/qiime2_results/diversity_raref_1192/alpha/chao1_tabulated.tsv", sep='\t')

# rename SampleID column
shannon <- rename(shannon, SampleID=X)
faith <- rename(faith, SampleID=X)
asv <- rename(asv, SampleID=X)
chao1 <- rename(chao1, SampleID=X)

# merge alpha diversity with metadata
metadata <- merge(metadata,shannon, by = c("SampleID"))
metadata <- merge(metadata,chao1, by = c("SampleID"))
metadata <- merge(metadata,asv, by = c("SampleID"))
metadata <- merge(metadata,faith, by = c("SampleID"))

# filter metadata
metadata <- filter(metadata, Genotype=="KO" | Genotype=="WT" | Genotype=="KO_IL17")
metadata <- filter(metadata, Experiment=="exp_1" | Experiment=="exp_2")
meta_wtko <- filter(metadata, Genotype=="KO" | Genotype=="WT")
meta_wtko17 <- filter(metadata, Genotype=="WT" | Genotype=="KO_IL17")
meta_koko17 <- filter(metadata, Genotype=="KO" | Genotype=="KO_IL17")


ERROR: Error: '\D' is an unrecognized escape in character string starting ""(C:Users\D"


## Survival depending on metadata

In [ ]:
# null model
cox.model <- coxph(Surv(Survival_day, Death)~1, data=metadata)
summary(cox.model)

In [ ]:
ggsurvplot(survfit(cox.model), data=metadata)

In [ ]:
# model genotype 3 groups
cox.model.geno <- coxph(Surv(Survival_day, Death)~Genotype, data=metadata)
summary(cox.model.geno)
# model genotype 2 groups (WT vs KO)
cox.model.geno.wtko <- coxph(Surv(Survival_day, Death)~Genotype, data=meta_wtko)
summary(cox.model.geno.wtko)
# model genotype 2 groups (KO vs KO+IL-17)
cox.model.geno.koko17 <- coxph(Surv(Survival_day, Death)~Genotype, data=meta_koko17)
summary(cox.model.geno.koko17)

In [ ]:
fit <- survfit(Surv(Survival_day, Death) ~ Genotype, data=metadata)
ggsurvplot(fit, data=metadata, conf.int=TRUE, pval=TRUE, pval.method=TRUE, tables.height=c(0.2, 0.2),
           risk.table=TRUE, cumevents=TRUE, legend.labs=c("KO", "KO + IL-17", "WT"))

In [ ]:
# model weight
cox.model.weight <- coxph(Surv(Survival_day, Death)~Weight_D3, data=metadata)
summary(cox.model.weight)

### Interpretation
*Předpokládám, že tyhle modely asi už máte, protože ty s mikrobiomem nesouvisí, ale aspoň jsem si vyzkoušela, jak to funguje :D*<br>
**Model phenotype**<br>
We found that survival does have a significant relationship with phenotype (WT vs KO). The model fits quite well given the Likelihood, Wald and Score tests all have p < 0.05.<br>
At a given instant of time the probability of dying for a KO mouse is 9.0 times higher than for a WT mouse.<br>
**Model weight**<br>
We found that survival does have a significant relationship with weight (p=0.00927). The model fits quite well given the Likelihood, Wald and Score tests all have p < 0.05.<br>
At a given instant of time the probability of dying for a mouse which weighs 1g less is 1.2 times higher than for a mouse which weighs 1g more.

## Survival depending on alpha diversity

In [ ]:
# model survival dependent on alpha diversity while adjusting for litter
cox.model.sha <- coxph(Surv(Survival_day, Death)~shannon_entropy, data=metadata)
summary(cox.model.sha)
cox.model.cha <- coxph(Surv(Survival_day, Death)~chao1, data=metadata)
summary(cox.model.cha)
cox.model.fai <- coxph(Surv(Survival_day, Death)~faith_pd, data=metadata)
summary(cox.model.fai)
cox.model.asv <- coxph(Surv(Survival_day, Death)~observed_features, data=metadata)
summary(cox.model.asv)

### Interpretation
**Shannon**<br>
This model tells us there is almost a significant relationship between Shannon entropy and survival (p=0.0552). Also we can be quite confident this model is a good fit, given the Likelihood, Wald and Score tests have in general low p value.<br>
At a given instant of time the probability of dying for a mouse which has 1 less point in Shannon entropy is 1.8 times higher than for a mouse with 1 point more in Shannon entropy while adjusting for litter effect.<br>
**Chao1**<br>
This model is not reliable at all given the Likelihood, Wald and Score tests  all have p=0.2. Additionally, it does not identify a significant relationship between Faith's PD and survival (p=0.173).<br>
**Faith's phylogenetic diversity**<br>
This model is not reliable at all given the Likelihood, Wald and Score tests  all have p=0.5. Additionally, it does not identify a significant relationship between Faith's PD and survival (p=0.528).<br>
**ASV**<br>
This model is not reliable at all given the Likelihood, Wald and Score tests  all have p=0.2. Additionally, it does not identify a significant relationship between Faith's PD and survival (p=0.175).<br>

In [ ]:
kruskal.test(shannon_entropy ~ Death_categ, data=metadata)

In [ ]:
options(repr.plot.width=8, repr.plot.height=8)

ggplot(metadata, aes(y=shannon_entropy, x=Death_categ, color=Death_categ)) +
      geom_boxplot() +
      geom_jitter(width=0.3, size=3, aes(shape=Litter)) +
      geom_signif(comparisons=list(c("died", "survived")), annotations=" p = 0.0478", textsize=5, color='black') +
      labs(x="", y="Shannon entropy") +
      theme_bw() +
      theme(panel.grid=element_blank(), legend.position="none",
            text=element_text(size=14, colour="black"),
            axis.text.x = element_text(size = 14, colour = "black"))

## Survival depending on beta diversity

In [ ]:
# load beta diversity pca coordinates
bray.pca <- read.delim("/Users/dagmarschierova/MEGA/IMG/study_5_revisions/qiime2_results/diversity_raref_1192_litter12/beta/bray_curtis_pca_exported.tsv", sep='\t')
jacc.pca <- read.delim("/Users/dagmarschierova/MEGA/IMG/study_5_revisions/qiime2_results/diversity_raref_1192_litter12/beta/jaccard_pca_exported.tsv", sep='\t')
uuni.pca <- read.delim("/Users/dagmarschierova/MEGA/IMG/study_5_revisions/qiime2_results/diversity_raref_1192_litter12/beta/unweighted_unifrac_pca_exported.tsv", sep='\t')
wuni.pca <- read.delim("/Users/dagmarschierova/MEGA/IMG/study_5_revisions/qiime2_results/diversity_raref_1192_litter12/beta/weighted_unifrac_pca_exported.tsv", sep='\t')

# add beta diversity PC1 to metadata
metadata$Bray_PC1 <- bray.pca$PC1
metadata$Jacc_PC1 <- jacc.pca$PC1
metadata$Uuni_PC1 <- uuni.pca$PC1
metadata$Wuni_PC1 <- wuni.pca$PC1
metadata$Bray_PC2 <- bray.pca$PC2
metadata$Jacc_PC2 <- jacc.pca$PC2
metadata$Uuni_PC2 <- uuni.pca$PC2
metadata$Wuni_PC2 <- wuni.pca$PC2

In [ ]:
# model survival dependent on beta diversity while adjusting for litter
cox.model.bray <- coxph(Surv(Survival_day, Death)~Bray_PC1+(1|Litter_num), data=metadata)
summary(cox.model.bray)
cox.model.jacc <- coxph(Surv(Survival_day, Death)~Jacc_PC1+(1|Litter_num), data=metadata)
summary(cox.model.jacc)
cox.model.uuni <- coxph(Surv(Survival_day, Death)~Uuni_PC1+(1|Litter_num), data=metadata)
summary(cox.model.uuni)
cox.model.wuni <- coxph(Surv(Survival_day, Death)~Wuni_PC1+(1|Litter_num), data=metadata)
summary(cox.model.wuni)

In [ ]:
# model survival dependent on beta diversity while adjusting for litter
cox.model2.bray <- coxph(Surv(Survival_day, Death)~Bray_PC1+Bray_PC2+(1|Litter_num), data=metadata)
summary(cox.model2.bray)
cox.model2.jacc <- coxph(Surv(Survival_day, Death)~Jacc_PC1+Jacc_PC2+(1|Litter_num), data=metadata)
summary(cox.model2.jacc)
cox.model2.uuni <- coxph(Surv(Survival_day, Death)~Uuni_PC1+Uuni_PC2+(1|Litter_num), data=metadata)
summary(cox.model2.uuni)
cox.model2.wuni <- coxph(Surv(Survival_day, Death)~Wuni_PC1+Wuni_PC2+(1|Litter_num), data=metadata)
summary(cox.model2.wuni)

In [ ]:
# copmarison of the models with one vs two predictors
anova.bray <- anova(cox.model.bray, cox.model2.bray, test="LRT")
anova.jacc <- anova(cox.model.jacc, cox.model2.jacc, test="LRT")
anova.uuni <- anova(cox.model.uuni, cox.model2.uuni, test="LRT")
anova.wuni <- anova(cox.model.wuni, cox.model2.wuni, test="LRT")
anova.bray
anova.jacc
anova.uuni
anova.wuni

### Interpretation
In this section we tested the survival dependence on beta diversity. In total, we tested two scenarios: **(1)** in the first scenario we used only the major principal coordinate axis (PC1) as a survival predictor, **(2)** in the second scenario we used two principal coordinate axes (PC1, PC2) as predictors. We compared these two scenarios using Likelihood ratio test and found out that using both PC axes results in slightly better models. Therefore, we decided to use the models with two predictors (PC1, PC2).<br>
**Bray-Curtis**<br>
This models tells us there is no significant relationship between first two PCoA axes (beta diversity) and survival. However, this model is not a good fit, given the Likelihood, Wald and Score tests all have p>0.05.<br>
**Jaccard**<br>
This models tells us there is no significant relationship between first two PCoA axes (beta diversity) and survival. However, this model is not a good fit, given the Likelihood, Wald and Score tests all have p>0.05.<br>
**unweighted UniFrac**<br>
This models tells us there is no significant relationship between PC1 and survival (p=0.084) while there is a significant relationship bwteen PC2 and survival (p=0.063). Also we can be quite confident this model is a good fit, given the Likelihood, Wald and Score tests all have p<0.05.<br>
**weighted UniFrac**<br>
This models tells us there is no significant relationship between first two PCoA axes (beta diversity) and survival. However, this model is not a good fit, given the Likelihood, Wald and Score tests all have p>0.05.<br>
Taken together, we have not found an effect of beta diversity on survival while controlling litter as a random effect.

## Survival depending on taxonomy

In [ ]:
# load taxonomy data
tax_data <- read_tsv("/Users/dagmarschierova/MEGA/IMG/study_5_revisions/qiime2_results/taxonomy_raref_1192/taxonomy-L6-abs.tsv", skip=1)
tax_id <- tax_data$`#OTU ID`
tax_data <- data.frame(tax_data[, -1], check.names = FALSE)
rownames(tax_data) <- tax_id
tax_data <- t(tax_data)

tax_data_rel <- read_tsv("/Users/dagmarschierova/MEGA/IMG/study_5_revisions/qiime2_results/taxonomy_raref_1192/taxonomy-L6-rel.tsv", skip=1)
tax_id <- tax_data_rel$`#OTU ID`
tax_data_rel <- data.frame(tax_data_rel[, -1], check.names = FALSE)
rownames(tax_data_rel) <- tax_id
tax_data_rel <- t(tax_data_rel)

# merge alpha diversity with metadata
tax_data <- merge(tax_data, metadata, by.x=0, by.y=c("SampleID"))
tax_wtko <- filter(tax_data, Genotype=="KO" | Genotype=="WT")
tax_wtko17 <- filter(tax_data, Genotype=="WT" | Genotype=="KO_IL17")
tax_koko17 <- filter(tax_data, Genotype=="KO" | Genotype=="KO_IL17")
tax_data_rel <- merge(tax_data_rel, metadata, by.x=0, by.y=c("SampleID"))


#### all 3 groups WT, KO, KO+IL-17

In [ ]:
# survival all 3 groups WT, KO, KO+IL-17

taxa <- seq_along(tax_data)[2:49]
pvals <- as.numeric()
exp_coef <- as.numeric()

# apply coxph model on each taxon and store pval and coeficient
options(warn=-1)
for (taxon in taxa)
{
  cox.model.tax <- coxph(Surv(tax_data$Survival_day, tax_data$Death)~tax_data[,taxon]+(1|tax_data$Litter_num))
  pvals[taxon]<-summary(cox.model.tax)$coefficients[,5]
  exp_coef[taxon]<-summary(cox.model.tax)$coefficients[,2]
  assign(paste0("model_", taxon), cox.model.tax)
}

# correct for multiple hypothesis testing
qval <- p.adjust(pvals, "BH")

# organize results in a table
table<-data.frame(Taxa=colnames(tax_data)[2:49], model_num=taxa, p_values=pvals[-1], BH_corr=qval[-1], exp_coef=exp_coef[-1])
write.table(table,"/Users/dagmarschierova/MEGA/IMG/study_5_revisions/statistic_results/survival_taxon.tsv",
            row.names = F, quote = F, sep="\t")
options(warn=0)

In [ ]:
kruskal_rum <- kruskal.test(`k__Bacteria;p__Firmicutes;c__Clostridia;o__Clostridiales;f__Ruminococcaceae;g__Ruminococcus` ~ Death_categ, data=tax_data)
kruskal_bla <- kruskal.test(`k__Bacteria;p__Firmicutes;c__Clostridia;o__Clostridiales;f__Lachnospiraceae;g__Blautia` ~ Death_categ, data=tax_data)
kruskal_xan <- kruskal.test(`k__Bacteria;p__Proteobacteria;c__Gammaproteobacteria;o__Xanthomonadales;f__Xanthomonadaceae;__` ~ Death_categ, data=tax_data)
kruskal_rum
kruskal_bla
kruskal_xan

In [ ]:
options(repr.plot.width=12, repr.plot.height=6)

ruminococcus <- ggplot(tax_wtko, aes(y=`k__Bacteria;p__Firmicutes;c__Clostridia;o__Clostridiales;f__Ruminococcaceae;g__Ruminococcus`,
                     x=Death_categ, color=Death_categ)) +
      geom_boxplot(outlier.shape=NA) +
      geom_jitter(width=0.3, height=0.2, size=3, aes(shape=Litter)) +
      labs(x="", y="Ruminococcus") +
      theme_bw() +
      theme(panel.grid=element_blank(), legend.position="none",
            text=element_text(size=14, colour="black"),
            axis.text.x = element_text(size=14, colour = "black"))

blautia <- ggplot(tax_wtko, aes(y=`k__Bacteria;p__Firmicutes;c__Clostridia;o__Clostridiales;f__Lachnospiraceae;g__Blautia`,
                     x=Death_categ, color=Death_categ)) +
      geom_boxplot(outlier.shape=NA) +
      geom_jitter(width=0.3, height=0.2, size=3, aes(shape=Litter)) +
      labs(x="", y="Blautia") +
      theme_bw() +
      theme(panel.grid=element_blank(), legend.position="none",
            text=element_text(size=14, colour="black"),
            axis.text.x = element_text(size=14, colour = "black"))

xanthomonadaceae <- ggplot(tax_wtko, aes(y=`k__Bacteria;p__Proteobacteria;c__Gammaproteobacteria;o__Xanthomonadales;f__Xanthomonadaceae;__`,
                     x=Death_categ, color=Death_categ)) +
      geom_boxplot(outlier.shape=NA) +
      geom_jitter(width=0.3, height=0.2, size=3, aes(shape=Litter)) +
      labs(x="", y="Xanthomonadaceae") +
      theme_bw() +
      theme(panel.grid=element_blank(), legend.position="none",
            text=element_text(size=14, colour="black"),
            axis.text.x = element_text(size=14, colour = "black"))

# combine plots
plt_tax_wtko <- ggarrange(ruminococcus, blautia, xanthomonadaceae,
                        ncol=3, nrow=1)
plt_tax_wtko

In [ ]:
write.table(tax_data,"/Users/dagmarschierova/MEGA/IMG/study_5_revisions/statistic_results/survival_data_L6-abs.tsv",
            row.names = F, quote = F, sep="\t")
write.table(tax_data_rel,"/Users/dagmarschierova/MEGA/IMG/study_5_revisions/statistic_results/survival_data_L6-rel.tsv",
            row.names = F, quote = F, sep="\t")